In [1]:
import pandas as pd
path_dict = "leandl_oesnpg_dicionario.parquet"
path_data = "leandl_oesnpg_dados.parquet"

dicionario_df = pd.read_parquet(path_dict)
dados_df = pd.read_parquet(path_data)

print("Dimensões do dicionário:", dicionario_df.shape)
print("Dimensões dos dados:", dados_df.shape)

import numpy as np
import pandas as pd

def to_fset(x):
    if isinstance(x, (list, tuple, np.ndarray)):
        return frozenset(x)
    if pd.isna(x):
        return frozenset()
    return frozenset([x])

df_temas = (
    dados_df.assign(palavras_chave=dados_df["palavras_chave"].apply(to_fset))
            [["tema_id", "tema", "uf_tema_info", "palavras_chave"]]
            .drop_duplicates()
)

df_temas = df_temas.reset_index(drop=True)

df_temas["tema"] = df_temas["tema"].str.lower()

dados_df["nome_producao"] = dados_df["nome_producao"].str.lower()
dados_df["descricao_resumo"] = dados_df["descricao_resumo"].str.lower()

Dimensões do dicionário: (25, 2)
Dimensões dos dados: (42046, 25)


In [2]:
from sentence_transformers import SentenceTransformer
from codecarbon import EmissionsTracker

models = {
  'bge-m3' : 'BAAI/bge-m3',
  'e5' : 'intfloat/multilingual-e5-large',
  'ibm' : 'ibm-granite/granite-embedding-278m-multilingual'
}

for m in models.keys():
  tracker = EmissionsTracker(project_name=m, log_level="error")
  tracker.start()
  
  model = SentenceTransformer(models[m])

  embeddings_tema = model.encode(df_temas.tema.to_list())

  df_temas['emb_tema'] = pd.Series(list(embeddings_tema), index=df_temas.index)

  for index,row in df_temas.iterrows(): df_temas.at[index,'embs_KW'] = 'x'
  for index,row in df_temas.iterrows():
    embs = []
    for keyword in row.palavras_chave:
      emb_k = model.encode(keyword)
      embs.append(np.mean([row.emb_tema,emb_k], axis=0))
    df_temas.at[index,'embs_KW'] = embs

  embeddings_CV = model.encode(dados_df.descricao_resumo)

  embeddings_titulo = model.encode(dados_df.nome_producao)

  meu_df = dados_df[['hash_id', 'tema_id', 'modelo_nivel']].copy()
  meu_df['emb_CV'] = pd.Series(list(embeddings_CV), index=meu_df.index)
  meu_df['emb_titulo'] = pd.Series(list(embeddings_titulo), index=meu_df.index)

  meu_df.to_pickle('df_' + m + '.pkl')
  df_temas.to_pickle('temas_' + m + '.pkl')

  emissions: float = tracker.stop()
  print(m, emissions)

bge-m3 0.011532880835854836
e5 0.008617280085026947
ibm 0.002706997490548545


In [ ]:
profile = dados_df.iloc[0].descricao_resumo

for m in models.keys():
  tracker = EmissionsTracker(project_name=m, log_level="error")
  tracker.start()

  model = SentenceTransformer(models[m])

  model.encode(profile)

  emissions: float = tracker.stop()
  print(m, emissions)

bge-m3 1.1575784184787513e-05
e5 1.1404820143553051e-05
ibm 1.29924123489197e-05


In [11]:
profile = dados_df.iloc[0].nome_producao

for m in models.keys():
  tracker = EmissionsTracker(project_name=m, log_level="error")
  tracker.start()

  model = SentenceTransformer(models[m])

  model.encode(str(profile))

  emissions: float = tracker.stop()
  print(m, emissions)

bge-m3 1.2382383373313724e-05
e5 1.1056905305404678e-05
ibm 1.1939663048901744e-05
